In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

#mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
#参数设置
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

n_input = 28
n_steps = 28
n_hidden = 128
n_classes = 10

In [3]:
tf.reset_default_graph()

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

x1 = tf.unstack(x, n_steps, 1)


In [4]:
stacked_rnn = []
stacked_bw_rnn = []
for i in range(3):
    stacked_rnn.append(tf.contrib.rnn.LSTMCell(n_hidden))
    stacked_bw_rnn.append(tf.contrib.rnn.LSTMCell(n_hidden))
mcell = tf.contrib.rnn.MultiRNNCell(stacked_rnn)
mcell_bw = tf.contrib.rnn.MultiRNNCell(stacked_bw_rnn)
outputs,_,_ = tf.contrib.rnn.stack_bidirectional_rnn([mcell],[mcell_bw],x1,dtype=tf.float32)

In [5]:
print(len(outputs),outputs[0].shape,outputs[1].shape)

pred = tf.contrib.layers.fully_connected(outputs[-1],n_classes,activation_fn = None)

28 (?, 256) (?, 256)


In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [7]:
# 启动session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算批次数据的准确率
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print (" Finished!")

    # 计算准确率 for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.984615, Training Accuracy= 0.30469
Iter 2560, Minibatch Loss= 1.644386, Training Accuracy= 0.46094
Iter 3840, Minibatch Loss= 1.241850, Training Accuracy= 0.58594
Iter 5120, Minibatch Loss= 1.097770, Training Accuracy= 0.63281
Iter 6400, Minibatch Loss= 0.772826, Training Accuracy= 0.73438
Iter 7680, Minibatch Loss= 0.794586, Training Accuracy= 0.74219
Iter 8960, Minibatch Loss= 0.546431, Training Accuracy= 0.83594
Iter 10240, Minibatch Loss= 0.651881, Training Accuracy= 0.77344
Iter 11520, Minibatch Loss= 0.554262, Training Accuracy= 0.83594
Iter 12800, Minibatch Loss= 0.547919, Training Accuracy= 0.85156
Iter 14080, Minibatch Loss= 0.434539, Training Accuracy= 0.85156
Iter 15360, Minibatch Loss= 0.470810, Training Accuracy= 0.82812
Iter 16640, Minibatch Loss= 0.405915, Training Accuracy= 0.88281
Iter 17920, Minibatch Loss= 0.392287, Training Accuracy= 0.87500
Iter 19200, Minibatch Loss= 0.220253, Training Accuracy= 0.93750
Iter 20480, Minibatch Loss= 0.21